# Interim Operational Capability 1

## Set up.

### Import packages.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

## Load data.

### The data are stored in a set of tab-separated value files in a folder.

In [3]:
designs = ty.Designs("data")

In [4]:
investments = ty.Investments("data")

### Compile the production and metric functions for each technology in the dataset.

In [5]:
designs.compile()

## Examine the data.

### The `functions` table specifies where the Python code for each technology resides.

In [6]:
designs.functions

,Style,Module,Capital,Fixed,Production,Metrics,Notes
Technology,,,,,,,
PV Generic,numpy,pv_residential_generic,capital_cost,fixed_cost,production,metrics,generic residential PV


### The `indices` table defines the subscripts for variables.

In [7]:
designs.indices.drop("Offset", axis = 1)

Description Notes
Technology Type    Index                                        
PV Generic Capital BoS                   balance of system      
                   Inverter               system inverters      
                   Module                    system module      
           Fixed   System                     whole system      
           Input   Strategic           strategic materials      
           Metric  Efficiency            module efficiency      
                   GHG                greenouse gas offset      
                   Hazardous      hazardous waste produced      
                   LCOC         levelized cost of capacity      
                   LCOE           levelized cost of energy      
                   Lifetime                module lifetime      
                   Strategic          strategic metal used      
                   Yield                    specific yield      
           Output  Electricity       electricity generated      
                   GHG               greenhouse gas offset      
                   Hazardous      hazardous waste produced

### The `designs` table contains the cost, input, efficiency, and price data for a scenario.

In [8]:
designs.designs

Value            Units  \
Technology Scenario  Variable          Index                                
PV Generic Reference Input             Strategic       0         g/system   
                     Input efficiency  Strategic       1            %/100   
                     Input price       Stategic        0              $/g   
                     Lifetime          BoS             1  system-lifetime   
                                       Inverter        1  system-lifetime   
                                       Module          1  system-lifetime   
                     Output efficiency Electricity     1            %/100   
                                       GHG             1            %/100   
                                       Hazardous       1            %/100   
                     Output price      Electricity     0            $/kWh   
                                       GHG             0          $/gCO2e   
                                       Hazardous       0              $/g   
                     Scale             NaN             1    system/system   

                                                                                              Notes  
Technology Scenario  Variable          Index                                                         
PV Generic Reference Input             Strategic                                    zero by default  
                     Input efficiency  Strategic                                         no scaling  
                     Input price       Stategic                                     zero by default  
                     Lifetime          BoS                                per-lifetime computations  
                                       Inverter                           per-lifetime computations  
                                       Module                             per-lifetime computations  
                     Output efficiency Electricity  see parameter table for individual efficiencies  
                                       GHG          see parameter table for individual efficiencies  
                                       Hazardous    see parameter table for individual efficiencies  
                     Output price      Electricity                   not tracking electricity price  
                                       GHG                             not tracking hazardous waste  
                                       Hazardous                       not tracking hazardous waste  
                     Scale             NaN                                               no scaling

### The `parameters` table contains additional techno-economic parameters for each technology.

In [9]:
designs.parameters.drop("Offset", axis = 1)

Value  \
Technology Scenario  Parameter                                                               
PV Generic Reference Customer Acquisition              st.triang(0.5, loc=2000, scale=0.2)   
                     Direct Labor                      st.triang(0.5, loc=2000, scale=0.2)   
                     Discount Rate                                                    0.07   
                     Greenhouse Gas Offset                                   -0.4490564e-3   
                     Hardware Capital                  st.triang(0.5, loc=80, scale=0.008)   
                     Hazardous Waste Content                                             0   
                     Insolation                                                       1000   
                     Installer Overhead & Profit    st.triang(0.5, loc=0.35, scale=3.5e-5)   
                     Inverter Capital                  st.triang(0.5, loc=0.3, scale=3e-5)   
                     Inverter Efficiency               st.triang(0.5, loc=0.9, scale=9e-5)   
                     Inverter Lifetime                st.triang(0.5, loc=16, scale=0.0016)   
                     Inverter Replacement              st.triang(0.5, loc=0.5, scale=5e-5)   
                     Location Capacity Factor          st.triang(0.5, loc=0.2, scale=2e-5)   
                     Module Aperture                   st.triang(0.5, loc=0.9, scale=9e-5)   
                     Module Capital                    st.triang(0.5, loc=110, scale=0.11)   
                     Module Degradation           st.triang(0.5, loc=0.0075, scale=7.5e-7)   
                     Module Efficiency              st.triang(0.5, loc=0.16, scale=1.6e-5)   
                     Module Lifetime                  st.triang(0.5, loc=25, scale=0.0025)   
                     Module O&M Fixed                  st.triang(0.5, loc=20, scale=0.002)   
                     Module Soiling Loss              st.triang(0.5, loc=0.05, scale=5e-6)   
                     Permitting                        st.triang(0.5, loc=600, scale=0.06)   
                     System Size                                                        36   

                                                      Units Notes  
Technology Scenario  Parameter                                     
PV Generic Reference Customer Acquisition          $/system   BCA  
                     Direct Labor                  $/system   BLR  
                     Discount Rate                   1/year    DR  
                     Greenhouse Gas Offset        gCO2e/kWh        
                     Hardware Capital                 $/m^2   BCC  
                     Hazardous Waste Content          g/m^2        
                     Insolation                       W/m^2   INS  
                     Installer Overhead & Profit          1   BOH  
                     Inverter Capital                   $/W   ICC  
                     Inverter Efficiency              %/100   IEF  
                     Inverter Lifetime                   yr   ILT  
                     Inverter Replacement             %/100   IRC  
                     Location Capacity Factor         %/100   MCF  
                     Module Aperture                  %/100   MAP  
                     Module Capital                   $/m^2   MCC  
                     Module Degradation                1/yr   MDR  
                     Module Efficiency                %/100   MEF  
                     Module Lifetime                     yr   MLT  
                     Module O&M Fixed               $/kW/yr   MOM  
                     Module Soiling Loss              %/100   MSL  
                     Permitting                    $/system   BPR  
                     System Size                        m^2   SSZ

### The `results` table specifies the units of measure for results of computations.

In [10]:
designs.results

Units Notes
Technology Variable Index                           
PV Generic Cost     Cost              $/system      
           Metric   Efficiency           %/100      
                    GHG          ΔgCO2e/system      
                    Hazardous            g/kWh      
                    LCOC                 $/Wdc      
                    LCOE                 $/kWh      
                    Lifetime                yr      
                    Strategic            g/kWh      
                    Yield                hr/yr      
           Output   Electricity     kWh/system      
                    GHG          ΔgCO2e/system      
                    Hazardous         g/system

### The `tranches` table specifies multually exclusive possibilities for investments: only one `Tranch` may be selected for each `Category`.

In [11]:
investments.tranches

,,,Amount,Notes
Category,Tranche,Scenario,,


### The `investments` table bundles a consistent set of tranches (one per category) into an overall investment.

In [12]:
investments.investments

,,,Notes
Investment,Category,Tranche,


## Evaluate the scenarios in the dataset.

In [13]:
scenario_results = designs.evaluate_scenarios(sample_count=50)

In [14]:
scenario_results.xs(1, level="Sample", drop_level=False)

Value          Units
Technology Scenario  Sample Variable Index                                    
PV Generic Reference 1      Cost     Cost          19539.025453       $/system
                            Metric   Efficiency        0.160011          %/100
                                     GHG              -0.000449  ΔgCO2e/system
                                     Hazardous         0.000000          g/kWh
                                     LCOC           3391.949136          $/Wdc
                                     LCOE              0.106125          $/kWh
                                     Lifetime         25.001477             yr
                                     Strategic         0.000000          g/kWh
                                     Yield          1278.394757          hr/yr
                            Output   Electricity  184112.901145     kWh/system
                                     GHG             -82.677077  ΔgCO2e/system
                                     Hazardous         0.000000       g/system

### Save results.

In [ ]:
scenario_results.to_csv("output/example-scenario.csv")

### Plot GHG metric.

In [ ]:
g = sb.boxplot(
    x="Scenario",
    y="Value",
    data=scenario_results.xs(
        ["Metric", "GHG"],
        level=["Variable", "Index"]
    ).reset_index()[["Scenario", "Value"]],
    order=[
        "2015 Actual"              ,
        "Module Slow Progress"      ,
        "Module Moderate Progress"  ,
        "Module Fast Progress"      ,
        "Inverter Slow Progress"    ,
        "Inverter Moderate Progress",
        "Inverter Fast Progress"    ,
        "BoS Slow Progress"         ,
        "BoS Moderate Progress"     ,
        "BoS Fast Progress"         ,
    ]
)
g.set(ylabel="GHG Reduction [gCO2e / system]")
g.set_xticklabels(g.get_xticklabels(), rotation=30);

### Plot LCOE metric.

In [ ]:
g = sb.boxplot(
    x="Scenario",
    y="Value",
    data=scenario_results.xs(
        ["Metric", "LCOE"],
        level=["Variable", "Index"]
    ).reset_index()[["Scenario", "Value"]],
    order=[
        "2015 Actual"              ,
        "Module Slow Progress"      ,
        "Module Moderate Progress"  ,
        "Module Fast Progress"      ,
        "Inverter Slow Progress"    ,
        "Inverter Moderate Progress",
        "Inverter Fast Progress"    ,
        "BoS Slow Progress"         ,
        "BoS Moderate Progress"     ,
        "BoS Fast Progress"         ,
    ]
)
g.set(ylabel="LCOE Reduction [USD / kWh]")
g.set_xticklabels(g.get_xticklabels(), rotation=30);

### Plot labor metric.

In [ ]:
g = sb.boxplot(
    x="Scenario",
    y="Value",
    data=scenario_results.xs(
        ["Metric", "Labor"],
        level=["Variable", "Index"]
    ).reset_index()[["Scenario", "Value"]],
    order=[
        "2015 Actual"              ,
        "Module Slow Progress"      ,
        "Module Moderate Progress"  ,
        "Module Fast Progress"      ,
        "Inverter Slow Progress"    ,
        "Inverter Moderate Progress",
        "Inverter Fast Progress"    ,
        "BoS Slow Progress"         ,
        "BoS Moderate Progress"     ,
        "BoS Fast Progress"         ,
    ]
)
g.set(ylabel="Labor Increase [USD / system]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

## Evaluate the investments in the dataset.

In [ ]:
investment_results = investments.evaluate_investments(designs, sample_count=50)

### Costs of investments.

In [ ]:
investment_results.amounts

### Benefits of investments.

In [ ]:
investment_results.metrics.xs(1, level="Sample", drop_level=False)

In [ ]:
investment_results.summary.xs(1, level="Sample", drop_level=False)

### Save results.

In [ ]:
investment_results.amounts.to_csv("output/residential_pv_multiobjective/example-investment-amounts.csv")

In [ ]:
investment_results.metrics.to_csv("output/residential_pv_multiobjective/example-investment-metrics.csv")

### Plot GHG metric.

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "GHG",
        level="Index"
    ).reset_index()[["Investment", "Value"]],
    order=[
        "Low R&D"   ,
        "Medium R&D",
        "High R&D"  ,
    ]
)
g.set(ylabel="GHG Reduction [gCO2e / system]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

### Plot LCOE metric.

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "LCOE",
        level="Index"
    ).reset_index()[["Investment", "Value"]],
    order=[
        "Low R&D"   ,
        "Medium R&D",
        "High R&D"  ,
    ]
)
g.set(ylabel="LCOE Reduction [USD / kWh]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

### Plot labor metric.

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "Labor",
        level="Index"
    ).reset_index()[["Investment", "Value"]],
    order=[
        "Low R&D"   ,
        "Medium R&D",
        "High R&D"  ,
    ]
)
g.set(ylabel="Labor Increase [USD / system]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

## Multi-objective decision analysis.

### Compute costs and metrics for tranches.

Tranches are atomic units for building investment portfolios. Evaluate all of the tranches, so we can assemble them into investments (portfolios).

In [ ]:
tranche_results = investments.evaluate_tranches(designs, sample_count=50)

Display the cost of each tranche.

In [ ]:
tranche_results.amounts

Display the metrics for each tranche.

In [ ]:
tranche_results.summary

Save the results.

In [ ]:
tranche_results.amounts.to_csv("output/residential_pv_multiobjective/example-tranche-amounts.csv")
tranche_results.summary.to_csv("output/residential_pv_multiobjective/example-tranche-summary.csv")

### Fit a response surface to the results.

The response surface interpolates between the discrete set of cases provided in the expert elicitation. This allows us to study funding levels intermediate between those scenarios.

In [ ]:
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Here are the categories of investment and the maximum amount that could be invested in each:

In [ ]:
evaluator.max_amount

Here are the metrics and their units of measure:

In [ ]:
evaluator.units

#### Example interpolation.

Let's evaluate the case where each category is invested in at half of its maximum amount.

In [ ]:
example_investments = evaluator.max_amount / 2
example_investments

In [ ]:
evaluator.evaluate(example_investments)

Let's evaluate the mean instead of outputing the whole distribution.

In [ ]:
evaluator.evaluate_statistic(example_investments, np.mean)

Here is the standard deviation:

In [ ]:
evaluator.evaluate_statistic(example_investments, np.std)

A risk-averse decision maker might be interested in the 10% percentile:

In [ ]:
evaluator.evaluate_statistic(example_investments, lambda x: np.quantile(x, 0.1))

### ε-Constraint multiobjective optimization

In [ ]:
optimizer = ty.EpsilonConstraintOptimizer(evaluator)

In order to meaningfully map the decision space, we need to know the maximum values for each of the metrics.

In [ ]:
metric_max = optimizer.max_metrics()
metric_max

#### Example optimization.

Limit spending to $3M.

In [ ]:
investment_max = 3e6

Require that the GHG reduction be at least 40 gCO2e/system and that the Labor wages not decrease.

In [ ]:
metric_min = pd.Series([40, 0], name = "Value", index = ["GHG", "Labor"])
metric_min

Compute the ε-constrained maximum for the LCOE.

In [ ]:
optimum = optimizer.maximize(
    "LCOE"                       ,
    total_amount = investment_max,
    min_metric   = metric_min    ,
    statistic    = np.mean       ,
)
optimum.exit_message

Here are the optimal spending levels:

In [ ]:
np.round(optimum.amounts)

Here are the three metrics at that optimum:

In [ ]:
optimum.metrics

*Thus, by putting all of the investment into Module R&D, we can expected to achieve a mean 3.75 ¢/kWh reduction in LCOE under the GHG and Labor constraints.*

It turns out that there is no solution for these constraints if we evaluate the 10th percentile of the metrics, for a risk-averse decision maker.

In [ ]:
optimum = optimizer.maximize(
    "LCOE"                       ,
    total_amount = investment_max,
    min_metric   = metric_min    ,
    statistic    = lambda x: np.quantile(x, 0.1),
)
optimum.exit_message

Let's try again, but with a less stringent set of constraints, only constraining GHG somewhat  but not Labor at all.

In [ ]:
optimum = optimizer.maximize(
    "LCOE"                                                         ,
    total_amount = investment_max                                  ,
    min_metric   = pd.Series([30], name = "Value", index = ["GHG"]),
    statistic    = lambda x: np.quantile(x, 0.1)                   ,
)
optimum.exit_message

In [ ]:
np.round(optimum.amounts)

In [ ]:
optimum.metrics

### Pareto surfaces.

##### Metrics constrained by total investment.

In [ ]:
pareto_amounts = None
for investment_max in np.arange(1e6, 9e6, 0.5e6):
    metrics = optimizer.max_metrics(total_amount = investment_max)
    pareto_amounts = pd.DataFrame(
        [metrics.values]                                         ,
        columns = metrics.index.values                           ,
        index   = pd.Index([investment_max / 1e6], name = "Investment [M$]"),
    ).append(pareto_amounts)
pareto_amounts

In [ ]:
sb.relplot(
    x         = "Investment [M$]",
    y         = "Value"          ,
    col       = "Metric"         ,
    kind      = "line"           ,
    facet_kws = {'sharey': False},
    data      = pareto_amounts.reset_index().melt(id_vars = "Investment [M$]", var_name = "Metric", value_name = "Value")
)

*We see that the LCOE metric saturates more slowly than the GHG and Labor ones.*

##### GHG vs LCOE, constrained by total investment.

In [ ]:
investment_max = 3
pareto_ghg_lcoe = None
for lcoe_min in 0.95 * np.arange(0.5, 0.9, 0.05) * pareto_amounts.loc[investment_max, "LCOE"]:
    optimum = optimizer.maximize(
        "GHG",
        max_amount   = pd.Series([0.9e6, 3.0e6, 1.0e6], name = "Amount", index = ["BoS R&D", "Inverter R&D", "Module R&D"]),
        total_amount = investment_max * 1e6                                 ,
        min_metric   = pd.Series([lcoe_min], name = "Value", index = ["LCOE"]),
    )
    pareto_ghg_lcoe = pd.DataFrame(
        [[investment_max, lcoe_min, optimum.metrics["LCOE"], optimum.metrics["GHG"], optimum.exit_message]],
        columns = ["Investment [M$]", "LCOE (min)", "LCOE", "GHG", "Result"]                               ,
    ).append(pareto_ghg_lcoe)
pareto_ghg_lcoe = pareto_ghg_lcoe.set_index(["Investment [M$]", "LCOE (min)"])
pareto_ghg_lcoe

In [ ]:
sb.relplot(
    x = "LCOE",
    y = "GHG",
    kind = "scatter",
    data = pareto_ghg_lcoe#[pareto_ghg_lcoe.Result == "Optimization terminated successfully."]
)

*The three types of investment are too decoupled to make an interesting pareto frontier, and we also need a better solver if we want to push to lower right.*

## Run the interactive explorer for the decision space.

<font color="red">Make sure the the `tk` package is installed on your machine.</font> Here is the Anaconda link: https://anaconda.org/anaconda/tk.

In [ ]:
w = ty.DecisionWindow(evaluator)
w.mainloop()

A new window should open that looks like the image below. Moving the sliders will cause a recomputation of the boxplots.

In [ ]:
Image("residential_pv_multiobjective_gui.png")